#### 공사비 기준 설비 카운팅

##### 학습대상 추출
* 공사비에 존재하는 공사번호
* 전선 갯 수: 1~11, 전주 갯 수: 1~10

In [1]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
from freeman.plt_setting import plt_settings
from freeman.aiddd.data_manager import read_data, write_data

# 차트 한글처리 지원
plt_settings()

#### 데이터 불러오기

In [2]:
_start_time = datetime.now()

# 제공받은 데이터: 자체적으로 3차에 받은 데이터
provided_data = {
    'line': read_data('LINE_DATA', data_type='provide', process_seq='3rd'),
    'pole': read_data('POLE_DATA', data_type='provide', process_seq='3rd'),
    'sl': read_data('SL_DATA', data_type='provide', process_seq='3rd')
}

# 2차에 받은 공사비 데이터를 가지고 자체적으로 4차에 전처리한 학습대상 데이터 
df_cons = read_data('1a-cons-training', data_type='preprocess', process_seq='4th')

print(
    f'Total Elapsed Time for Data Load: {datetime.now() - _start_time}\n'
    'Provided Data Shape:', end=' '
)
for data_type in provided_data.keys():
    print(f'{data_type} {provided_data[data_type].shape}', end=', ')
print(f'\nPreprocessing Data Shape: {df_cons.shape}')

Total Elapsed Time for Data Load: 0:00:48.229812
Provided Data Shape: line (40019, 77), pole (38533, 63), sl (22632, 57), 
Preprocessing Data Shape: (15335, 15)


#### 공사번호별 설비(전주/전선/인입선) 갯 수 계산

In [3]:
for data_type in provided_data.keys():
    df_current = provided_data[data_type]
    device_counts_per_cons_id = df_current['공사번호'].value_counts()
    column_name = f'{data_type}_cnts'
    
    df_cons = pd.merge(
        df_cons, device_counts_per_cons_id.rename(column_name),
        left_on='cons_id', right_on=device_counts_per_cons_id.index,
        how='left'
    )
    
    # 공사비에 있는 공사번호에 해당 설비가 없는 경우 NaN값이 들어감(그 값을 0으로)
    df_cons[column_name].fillna(0, inplace=True)

#### 중복으로 관리되는 전산화 번호 확인

In [4]:
provided_data['pole']['전산화번호'].value_counts()[:10]

전산화번호
4053W851    2
4189A521    2
4189A301    2
2663X353    2
3780R101    2
2663X052    2
6304S412    2
4053W872    2
2885D003    1
2582X841    1
Name: count, dtype: int64

In [5]:
df_pole = provided_data['pole']
duplicated_comp_id_rows = \
    df_pole[df_pole['전산화번호'].duplicated(keep=False)]

In [6]:
duplicated_comp_id_rows[['공사번호', '전산화번호']]

,공사번호,전산화번호
8631,476920213694,2663X052
8632,476920213694,2663X052
8635,476920213694,2663X353
8636,476920213694,2663X353
11084,477220203284,4053W851
11085,477220203284,4053W851
11086,477220203284,4053W872
11087,477220203284,4053W872
21589,474920213231,6304S412
22020,474920213966,6304S412


In [7]:
# 전체 공사비 데이터에서 해당 공사번호 레코드 추출
df_cons[df_cons.cons_id.isin(duplicated_comp_id_rows['공사번호'])]

,cons_id,total_cons_cost,cont_cap,last_mod_date,year,month,day,dayofweek,dayofyear,last_mod_eid,eid_code,eid_code_number,eid_number,office_name,office_number,line_cnts,pole_cnts,sl_cnts
8191,474920213231,3261098,3,2021-03-19 08:37:34,2021,3,19,4,78,2019108601,AAA,1,2019108601,제천지사,7,2.0,2.0,1.0
8389,474920213966,1647953,5,2021-07-06 12:40:48,2021,7,6,1,187,MDP2100070,MDP,8,2100070,제천지사,7,1.0,1.0,1.0


In [8]:
# 다행히 중복된 전산화번호가 포함된 공사번호는 1건 뿐임
# 전주, 전선, 인입선 데이터 더 적을 '474920213966' 공사번호 제거
df_cons = df_cons[df_cons.cons_id != '474920213966']

#### 학습대상 레코드 추출

In [9]:
MIN_POLE_LINE_COUNT = 0
MAX_POLE_COUNT = 10
MAX_LINE_COUNT = 11

training_data_conditions = \
    (df_cons.pole_cnts >  MIN_POLE_LINE_COUNT) & \
    (df_cons.pole_cnts <= MAX_POLE_COUNT) & \
    (df_cons.line_cnts >  MIN_POLE_LINE_COUNT) & \
    (df_cons.line_cnts <= MAX_LINE_COUNT)
    
# 불필요 레코드 갯 수
df_cons[~training_data_conditions].shape[0]

606

In [10]:
# 606개면 무시할 만 하니 제거
df_training = df_cons[training_data_conditions]

In [11]:
write_data('1b-merge-cons-device-counts-all', df_cons, process_seq='4th')
write_data('1b-merge-cons-device-counts-training', df_training, process_seq='4th')